## Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import seaborn as sns
from sklearn.preprocessing import StandardScaler

## Tratamento dos dados

In [2]:
df = pd.read_csv('/workspaces/Fuzzy_Clustering/datasets/haberman.csv')
df.columns = ["Age", "Op_year", "Axil_nodes", "Surv_status"]
df = df.rename(columns={"Surv_status": "Class"})
df["Class"].replace({1: 0, 2: 1}, inplace=True)
labels = df["Class"].values
df.drop("Class", axis=1, inplace=True)
dados = df.to_numpy()
dados

/tmp/ipykernel_15737/3128671020.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Class"].replace({1: 0, 2: 1}, inplace=True)


array([[30, 62,  3],
       [30, 65,  0],
       [31, 59,  2],
       [31, 65,  4],
       [33, 58, 10],
       [33, 60,  0],
       [34, 59,  0],
       [34, 66,  9],
       [34, 58, 30],
       [34, 60,  1],
       [34, 61, 10],
       [34, 67,  7],
       [34, 60,  0],
       [35, 64, 13],
       [35, 63,  0],
       [36, 60,  1],
       [36, 69,  0],
       [37, 60,  0],
       [37, 63,  0],
       [37, 58,  0],
       [37, 59,  6],
       [37, 60, 15],
       [37, 63,  0],
       [38, 69, 21],
       [38, 59,  2],
       [38, 60,  0],
       [38, 60,  0],
       [38, 62,  3],
       [38, 64,  1],
       [38, 66,  0],
       [38, 66, 11],
       [38, 60,  1],
       [38, 67,  5],
       [39, 66,  0],
       [39, 63,  0],
       [39, 67,  0],
       [39, 58,  0],
       [39, 59,  2],
       [39, 63,  4],
       [40, 58,  2],
       [40, 58,  0],
       [40, 65,  0],
       [41, 60, 23],
       [41, 64,  0],
       [41, 67,  0],
       [41, 58,  0],
       [41, 59,  8],
       [41, 5

## Método de agrupamento

In [3]:
class MFCM():
    def __init__(self, c, X, m):
        self.c = c
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.m = m

        ##
        self.global_var = np.var(X, axis=0).mean() # pré-calcula a variância global dos dados para regularização

    def initialize_u(self):
        u_flat = np.random.dirichlet(alpha=np.ones(self.c * self.p), size=self.n)
        return u_flat.reshape(self.n, self.c, self.p)
    
    def initialize_lambda(self):
        return np.ones((self.c, self.p))
    
    def find_centroides(self, X, U):
        u_m = U ** self.m
        numerador = np.sum(u_m * X[:, np.newaxis, :], axis=0)
        denominador = np.sum(u_m, axis=0)
        denominador = np.fmax(denominador, np.finfo(np.float64).eps)
        return numerador / denominador
    
    def get_distances(self, X, V):
        return (X[:, np.newaxis, :] - V[np.newaxis, :, :]) ** 2

    def update_u(self, D, Lambda):
        power = 1.0 / (self.m - 1)
        eps = np.finfo(np.float64).eps
        
        weighted_dist = D * Lambda
        weighted_dist = np.fmax(weighted_dist, eps) 
        
        term = weighted_dist ** (-power)
        
        denominator = np.sum(term, axis=(1, 2), keepdims=True)
        denominator = np.fmax(denominator, eps)
        
        return term / denominator
    
    def update_lambda(self, D, U):
        eps = np.finfo(np.float64).eps
        
        term_k = (U ** self.m) * np.fmax(D, eps)
        S_ij = np.sum(term_k, axis=0) 
        
        # Regularização Aditiva
        ##
        regularization_factor = 0.01 * self.global_var
        S_ij = S_ij + regularization_factor 
        
        prod_S = np.prod(S_ij, axis=1, keepdims=True)
        numerator = prod_S ** (1.0 / self.p)
        
        Lambda = numerator / S_ij
        
        return Lambda

    ##
    def calculate_objective_function(self, D, U, Lambda):
        term = (U ** self.m) * D
        sum_k = np.sum(term, axis=0)
        return np.sum(Lambda * sum_k)

## Clustering

In [4]:
def mfcm_run(dados, num_clusters, m=2, max_iter=1000, epsilon=1e-6):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(dados)
    
    mfcm = MFCM(c=num_clusters, X=X_scaled, m=m)
    
    indices = np.random.choice(X_scaled.shape[0], num_clusters, replace=False)
    centroids = X_scaled[indices]
    
    Lambda = np.ones((num_clusters, X_scaled.shape[1]))
    
    D = mfcm.get_distances(X_scaled, centroids)
    U = mfcm.update_u(D, Lambda)
    
    WARM_UP_ITERS = 20 

    for i in range(max_iter):
        U_old = U.copy()
        
        centroids = mfcm.find_centroides(X_scaled, U)
        D = mfcm.get_distances(X_scaled, centroids)
        
        ##
        if i >= WARM_UP_ITERS:
            Lambda = mfcm.update_lambda(D, U) 
        
        U = mfcm.update_u(D, Lambda)
        
        if np.linalg.norm(U - U_old) < epsilon:
            break
        
    J_final = mfcm.calculate_objective_function(D, U, Lambda)
    Delta = np.sum(U, axis=2)
    
    return centroids, U, Delta, Lambda, J_final

## Simulação de Monte Carlo

In [5]:
def calculate_objective_function(self, D, U, Lambda):
        term = (U ** self.m) * D
        sum_k = np.sum(term, axis=0)
        weighted_sum = Lambda * sum_k
        J = np.sum(weighted_sum)
        return J

In [9]:
def run_final_experiment(dados, labels, num_clusters=3, num_trials=100, restarts=50):
    results_ari = []
    results_ami = []
    
    for t in range(num_trials):
        best_J = np.inf
        best_pred = None
        
        for r in range(restarts):
            try:
                centroids, U, Delta, Lambda, J = mfcm_run(dados, num_clusters, m=2)
                
                if J > 1e-3 and J < best_J:
                    best_J = J
                    best_pred = np.argmax(Delta, axis=1)
            except:
                continue
        
        if best_pred is not None:
            ari = adjusted_rand_score(labels, best_pred)
            ami = adjusted_mutual_info_score(labels, best_pred)
            
            results_ari.append(ari)
            results_ami.append(ami)
            
            print(f"Trial {t+1}: J={best_J:.4f} | ARI={ari:.4f} | AMI={ami:.4f}")
        else:
            print(f"Trial {t+1}: Falha (Singularidade em todas as tentativas).")

    print(f"Mean ARI: {np.mean(results_ari):.4f} +/- {np.std(results_ari):.4f}")
    print(f"Mean AMI: {np.mean(results_ami):.4f} +/- {np.std(results_ami):.4f}")

In [10]:
run_final_experiment(dados, labels, num_clusters=2, num_trials=100, restarts=50)

Trial 1: J=0.2007 | ARI=0.0230 | AMI=-0.0004
Trial 2: J=0.1987 | ARI=-0.0192 | AMI=-0.0016
Trial 3: J=0.1947 | ARI=0.0073 | AMI=-0.0026
Trial 4: J=0.1948 | ARI=-0.0034 | AMI=-0.0031
Trial 5: J=0.1947 | ARI=0.0073 | AMI=-0.0026
Trial 6: J=0.1957 | ARI=0.0576 | AMI=0.0091
Trial 7: J=0.1942 | ARI=0.0095 | AMI=-0.0025
Trial 8: J=0.1968 | ARI=0.0084 | AMI=-0.0026
Trial 9: J=0.1967 | ARI=-0.0038 | AMI=-0.0030
Trial 10: J=0.1970 | ARI=0.0048 | AMI=-0.0029
Trial 11: J=0.1962 | ARI=0.0249 | AMI=-0.0001
Trial 12: J=0.1977 | ARI=-0.0028 | AMI=-0.0029
Trial 13: J=0.1977 | ARI=0.0085 | AMI=-0.0024
Trial 14: J=0.1982 | ARI=0.0048 | AMI=-0.0029
Trial 15: J=0.1981 | ARI=0.0048 | AMI=-0.0029
Trial 16: J=0.1947 | ARI=0.0073 | AMI=-0.0026
Trial 17: J=0.1947 | ARI=0.0073 | AMI=-0.0026
Trial 18: J=0.1948 | ARI=-0.0034 | AMI=-0.0031
Trial 19: J=0.1948 | ARI=-0.0034 | AMI=-0.0031
Trial 20: J=0.1948 | ARI=0.0368 | AMI=0.0026
Trial 21: J=0.1970 | ARI=0.0048 | AMI=-0.0029
Trial 22: J=0.1977 | ARI=0.0085 | AMI=-